# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
file = 'output_data/cities.csv'
cities = pd.read_csv(file)
cities.head()

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Nikolskoye,59.70,30.79,14.00,87,40,3.00,RU,1594408901
1,Vangaindrano,-23.35,47.60,20.09,90,88,4.15,MG,1594408901
2,Qaanaaq,77.48,-69.36,7.87,57,5,1.39,GL,1594408901
3,Albany,42.60,-73.97,28.89,75,100,3.74,US,1594408809
4,Sabha,27.04,14.43,32.04,18,0,5.35,LY,1594408902


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
gmaps.configure(api_key=g_key)
locations = cities[["Latitude", "Longitude"]].astype(float)
humidity = cities["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(
    locations, 
    weights=humidity, 
    dissipating=False, 
    max_intensity=100,
    point_radius = 1
    )

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
#Narrow down the cities to fit weather conditions.
ideal_weather = cities[(cities['Temperature'] <= 20) & (cities['Humidity'] < 50)].reset_index()
#Drop any rows will null values.
ideal_weather.dropna(inplace = True)

ideal_weather.head()

,index,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,27,Clyde River,70.47,-68.59,16.00,44,75,7.20,CA,1594408907
1,84,Haines Junction,60.75,-137.51,17.63,38,100,1.74,CA,1594408922
2,114,Oxford,51.75,-1.26,18.00,48,0,4.60,GB,1594408740
3,126,Ngunguru,-35.62,174.50,7.22,3,5,1.34,NZ,1594408932
4,179,Kokstad,-30.55,29.42,13.81,38,0,3.17,ZA,1594408945


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
#Store into variable named hotel_df.
hotel_df = ideal_weather

#Add a "Hotel Name" column to the DataFrame.
hotel_df['Hotel Name'] = ""
hotel_df['Hotel Lat'] = ""
hotel_df['Hotel Lng'] = ""

#Set parameters to search for hotels with 5000 meters.
radius = 5000

In [24]:
#Hit the Google Places API for each city's coordinates.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get location from df
    lat = row['Latitude']
    lng = row['Longitude'] 
    
    params = {
    "location": f'{lat},{lng}',
    "radius": radius,
    "type": "lodging",
    "key": g_key,
    }

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    #Store the first Hotel result into the DataFrame.j
    try:        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Hotel Lat'] = results[0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Hotel Lng'] = results[0]['geometry']['location']['lng']
        
    except (KeyError, IndexError):
        hotel_df.loc[index, 'Hotel Name'] = 'None found'
        pass

hotel_df.dropna(inplace = True)

In [32]:
#Plot markers on top of the heatmap.
gmaps.configure(api_key=g_key)
hotel_locations = hotel_df[["Hotel Lat", "Hotel Lng"]].astype(float)

In [33]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Hotel Lat", "Hotel Lng"]]

In [34]:
# Add marker layer ontop of heat map
hotel_markers = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)
fig.add_layer(hotel_markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))